In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tkinter import Tk, Label, Entry, Button, StringVar
import warnings

# Bỏ qua cảnh báo
warnings.filterwarnings("ignore", category=UserWarning)

# Đọc dữ liệu từ file CSV
data = pd.read_csv('data1.csv')
data['adjusted_ouput'] = data['output'] * (100 - data['abandonment_rate']) / 100

# Tính hệ số tương quan giữa adjusted_output và predicted_price
correlation_adjusted_output_price = data['adjusted_ouput'].corr(data['predicted_price'])

# Chuẩn bị dữ liệu cho mô hình
X = data[['product_price', 'nutri_glucid', 'nutri_protein', 'nutri_lipit', 'nutri_xenlulozo',
          'adjusted_ouput', 'storage_time', 'market_demand', 'competition',
          'tax_policy', 'profits']]
y = data['predicted_price']

# Chuẩn hóa dữ liệ
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# --- Hồi quy tuyến tính thủ công và với Gradient Descent ---
def linear_regression_manual(X, y):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
    return theta_best

def linear_regression_gradient_descent(X, y, learning_rate=0.01, n_iterations=1000):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    theta = np.random.randn(X_b.shape[1])
    for _ in range(n_iterations):
        gradients = (2 / X_b.shape[0]) * X_b.T.dot(X_b.dot(theta) - y)
        theta -= learning_rate * gradients
    return theta

# Hệ số hồi quy
theta_manual = linear_regression_manual(X_train, y_train)
theta_gd = linear_regression_gradient_descent(X_train, y_train)

def predict_manual(X, theta):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    return X_b.dot(theta)

# Hàm tính MSE
def calculate_mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

# Dự đoán và tính MSE trên tập kiểm tra
y_pred_manual = predict_manual(X_test, theta_manual)
y_pred_gd = predict_manual(X_test, theta_gd)
mse_manual = calculate_mse(y_test, y_pred_manual)
mse_gd = calculate_mse(y_test, y_pred_gd)
print("Manual Linear Regression MSE:", mse_manual)
print("Gradient Descent Linear Regression MSE:", mse_gd)

# --- Mô hình học máy ---
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "Decision Tree": DecisionTreeRegressor(max_depth=10),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=15),
    "Neural Network": MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=2000, learning_rate_init=0.001)
}

# Huấn luyện mô hình
trained_models = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    trained_models[name] = model

# Kiểm tra lỗi MSE của các mô hình
for name, model in trained_models.items():
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{name} MSE:", mse)

# --- Giao diện Tkinter ---
def predict_price():
    food_name = entry_food.get()
    if food_name in data['name'].values:
        food_data = data[data['name'] == food_name].drop(columns=['name', 'predicted_price'])

        input_data = food_data[['product_price', 'nutri_glucid', 'nutri_protein', 'nutri_lipit',
                                'nutri_xenlulozo', 'adjusted_ouput', 'storage_time',
                                'market_demand', 'competition', 'tax_policy', 'profits']].values
        input_data_scaled = scaler.transform(input_data)

        # Kết quả dự đoán từ các mô hình
        predictions = {name: model.predict(input_data_scaled)[0] for name, model in trained_models.items()}

        # Dự đoán với hồi quy tuyến tính thủ công và GD
        predictions["Manual Linear Regression"] = predict_manual(input_data_scaled, theta_manual)[0]
        predictions["GD Linear Regression"] = predict_manual(input_data_scaled, theta_gd)[0]

        # Hiển thị kết quả dự đoán
        result_str = f"Giá dự đoán cho {food_name}:\n"
        for name, price in predictions.items():
            result_str += f"{name}: {price:,.0f} VND\n"
        result.set(result_str)
    else:
        result.set("Không tìm thấy thực phẩm này.")

# Tạo giao diện Tkinter
root = Tk()
root.title("Dự đoán giá lương thực")

# Giao diện người dùng
Label(root, text="Nhập tên thực phẩm:").pack()
entry_food = Entry(root)
entry_food.pack()
result = StringVar()
Label(root, textvariable=result).pack()
Button(root, text="Dự đoán giá", command=predict_price).pack()

root.mainloop()